## 命名张量

In [11]:
import torch
from torchvision import transforms

batch_t = torch.randn(2,3,5,5) # B C H W
batch_t.shape

torch.Size([2, 3, 5, 5])

In [4]:
batch_t.shape

torch.Size([2, 3, 5, 5])

```python
"""
张量各维度的索引含义在程序中容易混淆，pytorch中提供了为张量维度命名的函数
"""
torch.tensor(,names = []) # 传参
_tensor.refine_names() # 能够添加名称但不改变现有名称
_tensor.rename() # 覆盖或删除(输入None)现有名称
```

In [7]:
batch_tnamed = batch_t.rename(...,'channels','rows','columns') # ...省略任意数量的维度
batch_tnamed.shape,batch_tnamed.names

(torch.Size([2, 3, 5, 5]), (None, 'channels', 'rows', 'columns'))

    接受维度的参数，也接受命名维度

In [12]:
weight_named = torch.tensor([0.2126,0.7152,0.0722],names = ('channels',))
img_t = torch.randn(3,5,5) 
Img_t = transforms.ToPILImage()(img_t)
Img_t.show()

In [14]:
img_gray = img_t.mean(-3) # 在channel维度上做平均取灰度图像
Img_gray = transforms.ToPILImage()(img_gray)
Img_gray.show()

In [15]:
img_named  = img_t.refine_names(...,'channels','rows','coloumns')
img_named.shape,img_named.names

(torch.Size([3, 5, 5]), ('channels', 'rows', 'coloumns'))

In [19]:
weight_aligned = weight_named.align_as(img_named) # 使用align_as()方法返回一个向量，添加缺失维度且将现有维度自动对齐
weight_aligned.shape,weight_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'coloumns'))

In [22]:
gray_named = (img_named*weight_aligned).sum('channels') # 使用维度名
gray_named.shape,gray_named.names

(torch.Size([5, 5]), ('rows', 'coloumns'))

## 使用dtype指定数字类型

    神经网络中发生的计算通常是float32精度执行的。这是由于一般，采用更高的精度如64位，并不会提高模型精度，反而需要更多的内存和计算时间。16位半精度浮点数类型在标准CPU中并不存在，而是在现代GPU提供。
    如有需求，可切换到半精度，其对精度影响也很小。

In [24]:
double_points  = torch.ones(10,2,dtype = torch.double)
short_points = torch.tensor([[1,2],[3,4]],dtype = torch.short)
short_points.dtype

torch.int16

In [25]:
double_points = torch.ones(10,2).double() # 其他方式
double_points.dtype

torch.float64

In [26]:
double_points = torch.zeros(2,2).to(torch.double) # to() 方法会检测转换是否必要
double_points.dtype

torch.float64

## transpose() 函数

In [29]:
tmp = torch.ones(2,3)
tmp_t = torch.transpose(tmp,0,1)
tmp_t1 = tmp.transpose(0,1)
tmp_t.shape,tmp_t1.shape

(torch.Size([3, 2]), torch.Size([3, 2]))

## 张量的存储视图

In [53]:
points = torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points.storage() # 访问给定张量的存储区

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [32]:
points_storage = points.storage()
points_storage[0] # 通过存储区访问张量

4.0

    修改张量中的存储值有很多方法，值得注意的是，是否对输入张量对象进行更改根据下划线判断

In [34]:
a = torch.ones(2,3)
a.zero_()
a

tensor([[0., 0., 0.],
        [0., 0., 0.]])

## 张量元数据：大小、偏移量、步长

In [36]:
points = torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points.stride() # stride[0]表示跳行所需的步长，stride[1]存储列步长

(2, 1)

### 无复制转置

In [43]:
"""
使用.t()实现无复制转置
"""
points = torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [44]:
points_t = points.t()
points_t


tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [45]:
id(points.storage()) == id(points_t.storage()) # 检测相同存储区内


True

### 高维转置

In [47]:
a = torch.ones(3,4,5)
a_t = a.transpose(0,2) # 指定需要转置的维度
id(a.storage()) == id(a_t.storage())

True

In [48]:
a.stride(),a_t.stride()

((20, 5, 1), (1, 5, 20))

###  连续张量

使用`contiguous()`方法可以查看并排列为连续张量，原理为更改存储及步长，不更改张量内容

In [49]:
points.is_contiguous(),points_t.is_contiguous()

(True, False)

In [50]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [51]:
points_t.contiguous().storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

## 将张量存储到GPU

采用两种方法：
- 创建时声明：`points_gpu = torch.tensor(,device = 'cuda')`
- `points_gpu = points.to(device = 'cuda')`

## Tensor 与 Numpy 互操作性

### 从张量得到Numpy数组

In [54]:
points = torch.ones(3,4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

### 从Numpy到Tensor

In [55]:
points = torch.from_numpy(points_np)
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

## 张量的保存及读取

通过以下方法可以将`points`保存到`ourpoints.t`文件中：
```python
torch.save(points,'../data/oourpoints.t')

with open('../data/ourpoints.t','wb') as f:
    torch.save(points,f)
```
同样，对张量的加载也可以实现
```python
points = torch.load('../data/ourpoints.t')

with open('../data/ourpoints.t','rb') as f:
    points = torch.load(f)
```

## 练习题

In [61]:
a = torch.tensor(list(range(9)))
a.shape,a.storage_offset(),a.stride()

(torch.Size([9]), 0, (1,))

In [69]:
b = a.view(3,3) # 将a通过更改步长变为目标格式，并返回新张量
b,id(a.storage()) == id(b.storage()),b.stride()

(tensor([[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]),
 True,
 (3, 1))